In [121]:
import numpy as np
import scipy.ndimage as sciim
import h5py

from importlib import reload
import utils
utils = reload(utils)

import plotting_tools as pt
pt = reload(pt)

import matplotlib.pyplot as plt
%matplotlib inline

In [164]:
base_fn = "output_travelling_vortex"
directory = "output_travelling_vortex"
py_directory = "../%s/" %directory

Nx = 32
Ny = 32
et = 6.0
N = 1

time = 0.0
# time1 = [0.0,0.25,0.50,0.75,1.0]
time1 = np.arange(0.0,6.05,0.05)

period = np.arange(0.0,1.05,0.05)

# time1 = [0.0,0.0]
attributes = ['rho','rhou','rhov','rhoY','p2_nodes']

steps_c = np.arange(0.0,float(Nx)*et+.1,(float(Nx)-0.0)/(time1.shape[0]-1))
steps_n = np.arange(0.0,(float(Nx)+1.0)*et+0.1,(float(Nx)+0.0)/(time1.shape[0]-1))

steps_c = np.arange(0.0,float(Nx)*et+0.05,float(Nx)*0.05)
steps_n = np.arange(0.0,float(Nx+1)*et+0.05,float(Nx+1)*0.05)

tc = utils.test_case(base_fn,py_directory,Nx,Ny,et)
fn = tc.get_filename(N,'comp_bal')
path = tc.get_path(fn)

debug = False

for nn, stp in enumerate(steps_c):
    for attribute in attributes:
        if attribute == 'p2_nodes':
            stp = steps_n[nn]
            
        comp0 = tc.get_arr(path, time, N, attribute, label_type='TIME', tag='after_full_step', inner=True, avg=False)
        comp0 = comp0[0]
        comp0 = np.pad(comp0,[0,1],mode='wrap')

        comp1 = tc.get_arr(path, time1[nn], N, attribute, label_type='TIME', tag='after_full_step', inner=False, avg=False)
        comp1 = comp1[0]

        shifted = sciim.shift(comp0, stp, mode='wrap')
        shifted = shifted[:-1,:-1]
        shifted = np.pad(shifted,2,mode='wrap')

        if debug == True:
            if nn % 10 == 0:
                diff = [shifted-comp1,"diff, %.2f" %time1[nn]]
                shifted = [shifted,"shifted"]
                comp1 = [comp1,"comp1"]

                pl = pt.plotter([shifted,comp1,diff],ncols=3)
                pl.plot(inner=True)
        else:
            file = h5py.File("output_travelling_vortex_ensemble=%i_%i_%i_%.1f_truthgen.h5" %(N,Nx,Ny,et))
            name = "ensemble_mem=0_%.3f_after_full_step" %time1[nn]
            file.create_dataset(str(attribute) + '/' + str(attribute) + '_' + str(name), data=shifted, chunks=True, compression='gzip', compression_opts=4, dtype=np.float32)
            file.close()

OSError: Unable to open file (unable to open file: name = '../output_travelling_vortex/output_travelling_vortex_ensemble=1_32_32_6.0_comp_bal.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)